<a href="https://colab.research.google.com/github/LLMUSER/on-device-tests/blob/main/Converting_Distill_Bert_SQUAD_to__mlmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install coremltools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.8 MB/s eta 0:00:00


In [4]:
import torch
import numpy as np

In [18]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')
# print(model)
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 1.1810, -0.4073,  0.9986,  ..., -0.7445,  0.0380, -0.5510],
         [ 1.6172, -0.6785,  1.6932,  ..., -0.8216, -0.2387, -0.6187],
         [ 2.0840, -0.5496,  1.3313,  ..., -0.7791,  0.1698, -0.3950],
         ...,
         [ 0.2879, -0.1813,  1.2631,  ..., -0.2022,  0.4699,  0.5535],
         [ 0.6069, -0.1943,  0.7584,  ..., -0.5106, -0.4027, -0.4910],
         [ 1.0183, -0.8215,  0.9088,  ..., -0.8094,  0.8372, -0.2027]]]), hidden_states=None, attentions=None)


In [5]:
print(inputs)

{'input_ids': tensor([[  101,  2627,  1108,  3104,  1124, 15703,   136,   102,  3104,  1124,
         15703,  1108,   170,  3505, 16797,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [10]:
#Creating a wrapper class for the model
import torch.nn as nn
class DistillWrapper(nn.Module):
  def __init__(self):
    super(DistillWrapper, self).__init__()
    self.model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')

  def forward(self ,x ):
    res = self.model(x)
    return res['last_hidden_state']

traceable_model = DistillWrapper().eval()
print(traceable_model)

DistillWrapper(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(

In [15]:
trace_model = torch.jit.trace(traceable_model , inputs['input_ids']+inputs['attention_mask'])

/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [16]:
import coremltools as ct
mlmodel_1 = ct.convert(
    trace_model,
    convert_to='neuralnetwork',
    inputs=[ct.TensorType(name="input_ids", shape=(1,ct.RangeDim(1, 768)))],
)

Running MIL default pipeline:  15%|█▌        | 10/65 [00:00<00:01, 42.52 passes/s]/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:267: UserWarning: Output, '469', of the source model, has been renamed to 'var_469' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 385/385 [00:10<00:00, 36.32 ops/s]


In [17]:
mlmodel_1.save("DistillBertFinetunedSquad.mlmodel")